In [281]:
import pandas as pd
import numpy as np
import numpy.linalg as la
import pickle
from pykalman import KalmanFilter

In [282]:
# def add_gaussian_noise_to_signal(signal, mean=0, std=1):
#     noisy_signal = signal.copy()
#     noisy_signal['depth'] = noisy_signal['depth'] + np.random.normal(mean, std, len(noisy_signal['depth']))
#     return noisy_signal
# 
# df['noisy_signal'] = df['decrease'].apply(lambda x: add_gaussian_noise_to_signal(x, mean=0, std=0.1))  # Adjust `std` as needed

In [283]:
df = pd.read_pickle("flood_df")
df.head()

,deployment_id,label,inflection_t,signal,signal_padded,signal_sim,signal_sim_padded
uuid,,,,,,,
1690089,daily_happy_satyr,flood,6961,"{'time': [0, 567, 1009, 1639, 1765, 2080, 2395...","{'time': [0, 60, 120, 180, 240, 300, 867, 1309...","{'time': [0, 567, 1009, 1639, 1765, 2080, 2395...","{'time': [0, 60, 120, 180, 240, 300, 867, 1309..."
2578925,daily_happy_satyr,flood,7062,"{'time': [0, 1889, 1952, 2960, 4914, 4921, 674...","{'time': [0, 60, 120, 180, 240, 300, 2189, 225...","{'time': [0, 1889, 1952, 2960, 4914, 4921, 674...","{'time': [0, 60, 120, 180, 240, 300, 2189, 225..."
5301386,daily_happy_satyr,flood,5303,"{'time': [0, 1323, 1898, 1906, 2409, 2535, 278...","{'time': [0, 60, 120, 180, 240, 300, 1623, 219...","{'time': [0, 1323, 1898, 1906, 2409, 2535, 278...","{'time': [0, 60, 120, 180, 240, 300, 1623, 219..."
2808962,daily_happy_satyr,flood,4789,"{'time': [126, 441, 503, 566, 882, 1008, 1071,...","{'time': [0, 60, 120, 180, 240, 300, 615, 677,...","{'time': [126, 441, 503, 566, 882, 1008, 1071,...","{'time': [0, 60, 120, 180, 240, 300, 615, 677,..."
9041605,daily_happy_satyr,flood,6188,"{'time': [0, 379, 1449, 1575, 1890, 2394, 2709...","{'time': [0, 60, 120, 180, 240, 300, 679, 1749...","{'time': [0, 379, 1449, 1575, 1890, 2394, 2709...","{'time': [0, 60, 120, 180, 240, 300, 679, 1749..."


In [284]:
def preprocess_signal(signal):
    time = signal['time']
    depth = signal['depth']
    return pd.DataFrame({'time': time, 'depth': depth}).sort_values(by='time')

df['processed_signal'] = df['signal_padded'].apply(preprocess_signal)
print(df['processed_signal'].iloc[0])

     time  depth
0       0    0.0
1      60    0.0
2     120    0.0
3     180    0.0
4     240    0.0
5     300    4.0
6     867   18.0
7    1309   25.0
8    1939   35.0
9    2065   37.0
10   2380   50.0
11   2695   64.0
12   2947   79.0
13   3010   82.0
14   3136   90.0
15   3387  110.0
16   3766  139.0
17   4018  160.0
18   4080  164.0
19   4332  181.0
20   5215  237.0
21   5781  266.0
22   6033  274.0
23   6229  283.0
24   6489  292.0
25   6560  296.0
26   6568  296.0
27   6694  298.0
28   6757  298.0
29   6820  300.0
30   7198  307.0
31   7261  308.0
32   7386  310.0
33   7575  307.0
34   7638  302.0
35   8142  297.0
36   8269  292.0
37   9088  268.0
38   9151  266.0
39   9403  260.0
40  10033  240.0
41  10536  220.0
42  10915  204.0
43  11230  189.0
44  11734  162.0
45  12238  131.0
46  12868   85.0
47  12993   74.0
48  13182   59.0
49  13497   34.0
50  13560   24.0
51  13812    2.0
52  13872    0.0
53  13932    0.0
54  13992    0.0
55  14052    0.0
56  14112    0.0


In [285]:
def vectorized_kalman_filter_3d(signal):
    observations = signal['depth'].values
    times = signal['time'].values
    time_diffs = np.diff(times, prepend=times[0])  # Compute time gaps with 0 for the first element

    # Initialize state and covariance matrices
    n_observations = len(observations)
    state_dim = 3  # [depth, velocity, acceleration]
    filtered_means = np.zeros((n_observations, state_dim))
    smoothed_means = np.zeros((n_observations, state_dim))
    covariances = np.zeros((n_observations, state_dim, state_dim))

    # Initial state and covariance
    state_mean = np.array([observations[0], 0, 0])  # Initial [depth, velocity, acceleration]
    state_covariance = np.eye(state_dim) * 1  # Initial uncertainty
    observation_matrix = np.array([[1, 0, 0]])  # Observation model
    observation_covariance = np.array([[1]])  # Observation noise
    process_noise_base = np.array([[6, 0.05, 0.0], [6, 0.1, 0.0], [0.0, 0.0, 0.0]]) / 10

    epsilon = 1e-6  # Small value for regularization

    # Filtering
    for t in range(n_observations):
        if t > 0:
            delta_t = time_diffs[t - 1]
            transition_matrix = np.array([
                [1, delta_t, 0.5 * delta_t**2],
                [0, 1, delta_t],
                [0, 0, 1]
            ])
            process_noise = process_noise_base * delta_t

            # Predict step
            predicted_state_mean = np.dot(transition_matrix, state_mean)
            predicted_state_cov = (
                np.dot(transition_matrix, np.dot(state_covariance, transition_matrix.T)) + process_noise
            )

            # Update step
            innovation = observations[t] - np.dot(observation_matrix, predicted_state_mean)
            innovation_cov = (
                np.dot(observation_matrix, np.dot(predicted_state_cov, observation_matrix.T))
                + observation_covariance
            )
            kalman_gain = np.dot(
                predicted_state_cov,
                np.dot(observation_matrix.T, np.linalg.inv(innovation_cov + epsilon * np.eye(innovation_cov.shape[0])))
            )
            state_mean = predicted_state_mean + np.dot(kalman_gain, innovation)
            state_covariance = predicted_state_cov - np.dot(
                kalman_gain, np.dot(observation_matrix, predicted_state_cov)
            )

        # Store filtered results
        filtered_means[t] = state_mean
        covariances[t] = state_covariance

    # Smoothing
    smoothed_means[-1] = filtered_means[-1]
    smoothed_covariance = covariances[-1]
    for t in range(n_observations - 2, -1, -1):
        delta_t = time_diffs[t]
        transition_matrix = np.array([
            [1, delta_t, 0.5 * delta_t**2],
            [0, 1, delta_t],
            [0, 0, 1]
        ])

        # RTS smoother gain
        predicted_covariance = (
            np.dot(transition_matrix, np.dot(covariances[t], transition_matrix.T)) + process_noise_base * delta_t
        )
        predicted_covariance += epsilon * np.eye(predicted_covariance.shape[0])  # Regularization
        smoother_gain = np.dot(
            covariances[t],
            np.dot(transition_matrix.T, np.linalg.pinv(predicted_covariance))  # Use pseudo-inverse
        )

        # Update smoothed state
        smoothed_means[t] = (
            filtered_means[t]
            + np.dot(smoother_gain, (smoothed_means[t + 1] - np.dot(transition_matrix, filtered_means[t])))
        )

    # Add filtered and smoothed results to the signal DataFrame
    signal['kalman_depth'] = filtered_means[:, 0]
    signal['smoothed_depth'] = smoothed_means[:, 0]
    signal['velocity'] = filtered_means[:, 1]
    signal['acceleration'] = filtered_means[:, 2]
    return signal


In [286]:
from pykalman import KalmanFilter
import numpy as np

def em_kalman_global(processed_signals, max_iterations=50, tol=1e-4):
    """
    Perform EM to estimate global parameters across multiple time-series signals.
    
    Args:
        processed_signals: A Series where each element is a time-series DataFrame (e.g., df['processed_signal']).
        max_iterations: Maximum number of EM iterations.
        tol: Convergence tolerance.

    Returns:
        A tuple of global parameters:
        - transition_matrix: Estimated state transition matrix.
        - process_noise_base: Estimated process noise covariance.
        - observation_covariance: Estimated observation noise covariance.
    """
    # Normalize the signals without modifying the originals
    signals_normalized = []
    signal_scales = []
    for signal in processed_signals:
        signal_copy = signal.copy()  # Work on a copy to preserve original data
        scale = np.std(signal_copy['depth'].values)
        if scale == 0:
            scale = 1  # Avoid division by zero for constant signals
        signal_copy['depth'] /= scale  # Normalize depth
        signals_normalized.append(signal_copy)
        signal_scales.append(scale)

    # Initial global parameters
    state_dim = 3
    observation_dim = 1
    transition_matrix = np.array([
        [1, 1, 0.5],
        [0, 1, 1],
        [0, 0, 1]
    ])
    observation_matrix = np.array([[1, 0, 0]])
    process_noise_base = np.eye(state_dim) * 0.1
    observation_covariance = np.eye(observation_dim) * 0.1
    epsilon = 1e-6  # Small value for regularization

    for iteration in range(max_iterations):
        # Initialize accumulators for sufficient statistics
        total_transition_numerator = np.zeros_like(transition_matrix)
        total_transition_denominator = np.zeros((state_dim, state_dim))
        total_process_noise = np.zeros_like(process_noise_base)
        total_observation_covariance = np.zeros_like(observation_covariance)

        for signal, scale in zip(signals_normalized, signal_scales):
            # Extract observations
            observations = signal['depth'].values
            n_observations = len(observations)

            # Initialize Kalman Filter
            kf = KalmanFilter(
                transition_matrices=transition_matrix,
                observation_matrices=observation_matrix,
                transition_covariance=process_noise_base,
                observation_covariance=observation_covariance,
                initial_state_mean=np.zeros(state_dim),
                initial_state_covariance=np.eye(state_dim)
            )

            # Apply filtering and smoothing
            smoothed_state_means, smoothed_state_covariances = kf.smooth(observations)

            # Accumulate statistics for M-step
            for t in range(1, n_observations):
                residual = smoothed_state_means[t] - np.dot(transition_matrix, smoothed_state_means[t - 1])
                total_transition_numerator += np.outer(smoothed_state_means[t], smoothed_state_means[t - 1])
                total_transition_denominator += np.outer(smoothed_state_means[t - 1], smoothed_state_means[t - 1])
                total_process_noise += np.outer(residual, residual)

            for t in range(n_observations):
                innovation = observations[t] - np.dot(observation_matrix, smoothed_state_means[t])
                total_observation_covariance += np.outer(innovation, innovation)

        # M-Step: Update global parameters
        transition_matrix = np.dot(total_transition_numerator, np.linalg.pinv(total_transition_denominator))
        process_noise_base = total_process_noise / (len(processed_signals) * (n_observations - 1))
        observation_covariance = total_observation_covariance / (len(processed_signals) * n_observations)

        # Regularize covariance updates
        observation_covariance += epsilon * np.eye(observation_dim)
        observation_covariance = np.clip(observation_covariance, 0, 10)  # Cap values for stability

        # Convergence check
        if np.max(np.abs(total_transition_numerator - transition_matrix)) < tol:
            break


    return transition_matrix, process_noise_base, observation_covariance


In [287]:
transition_matrix, process_noise_base, observation_covariance = em_kalman_global(df['processed_signal'])
global_params = {'transition_matrix': transition_matrix, 'process_noise_base':process_noise_base, 'observation_covariance':observation_covariance}

In [288]:
print(global_params)

{'transition_matrix': array([[ 9.99018274e-01,  9.48795654e-01, -1.43292859e-01],
       [ 1.09130675e-03,  9.89511485e-01,  1.83790053e+00],
       [-1.82489161e-05, -3.37200636e-03,  9.69293568e-01]]), 'process_noise_base': array([[ 1.97062489e-17,  3.51189104e-21, -8.38193436e-21],
       [ 3.51189104e-21,  1.11570011e-23,  3.71038592e-24],
       [-8.38193436e-21,  3.71038592e-24,  6.67319294e-24]]), 'observation_covariance': array([[7.07154092]])}


In [289]:
with open('global_params.pkl', 'wb') as file:
    pickle.dump(global_params, file)

In [290]:
def apply_kalman_filter(processed_signals, transition_matrix, process_noise_base, observation_covariance):
    """
    Apply the Kalman Filter and Smoother using global parameters to all processed signals.

    Args:
        processed_signals: A Series where each element is a time-series DataFrame (e.g., df['processed_signal']).
        transition_matrix: The global state transition matrix estimated from EM.
        process_noise_base: The global process noise covariance estimated from EM.
        observation_covariance: The global observation noise covariance estimated from EM.

    Returns:
        A DataFrame with the original signal and added columns for filtered and smoothed results.
    """
    results = []

    for signal in processed_signals:
        signal_copy = signal.copy()  # Preserve the original data
        observations = signal_copy['depth'].values

        # Initialize Kalman Filter with the global parameters
        kf = KalmanFilter(
            transition_matrices=transition_matrix,
            observation_matrices=np.array([[1, 0, 0]]),  # Observation model
            transition_covariance=process_noise_base,
            observation_covariance=observation_covariance,
            initial_state_mean=np.zeros(transition_matrix.shape[0]),
            initial_state_covariance=np.eye(transition_matrix.shape[0])
        )

        # Apply filtering and smoothing
        filtered_state_means, _ = kf.filter(observations)
        smoothed_state_means, _ = kf.smooth(observations)

        # Add filtered and smoothed results to the signal DataFrame
        signal_copy['filtered_depth'] = filtered_state_means[:, 0]  # First state: depth
        signal_copy['smoothed_depth'] = smoothed_state_means[:, 0]  # First state: depth
        signal_copy['velocity'] = filtered_state_means[:, 1]  # Second state: velocity
        signal_copy['acceleration'] = filtered_state_means[:, 2]  # Third state: acceleration

        results.append(signal_copy)

    return results

In [291]:
filtered_and_smoothed_signals = apply_kalman_filter(
    df['processed_signal'], 
    transition_matrix, 
    process_noise_base, 
    observation_covariance
)


In [295]:
print(filtered_and_smoothed_signals[5])

    time  depth  filtered_depth  smoothed_depth  velocity  acceleration
0      0    0.0        0.000000      -14.222196  0.000000      0.000000
1     60    0.0        0.000000       -5.869715  0.000000      0.000000
2    120    0.0        0.000000        2.189674  0.000000      0.000000
3    180    0.0        0.000000        9.922296  0.000000      0.000000
4    240    0.0        0.000000       17.297464  0.000000      0.000000
5    300   10.0        7.104479       24.287543  4.502586      0.477264
6    740   15.0       13.741252       30.867991  6.655540      0.566013
7    866   18.0       18.758472       37.017386  7.075441      0.481748
8   1001   19.0       21.731063       42.717428  6.384731      0.338724
9   1574   20.0       23.584270       47.952933  5.419709      0.213516
10  2014   25.0       26.825993       52.711801  5.157691      0.155129
11  2141   28.0       29.934264       56.984973  4.884700      0.107396
12  2715   42.0       37.858592       60.766376  6.000095      0

In [293]:
df['filtered_and_smoothed_signal'] = filtered_and_smoothed_signals
    # transition_matrix = initial_guess['transition_matrix']
    # process_noise_cov = initial_guess['process_noise_cov']
    # observation_matrix = initial_guess['observation_matrix']
    # observation_noise_cov = initial_guess['observation_noise_cov']